In [1]:
from pprint import pprint
from types import SimpleNamespace
import anomapy.train.sssn as sssn
import pyworld.toolkit.tools.wbutils as wbu
import datasets
import numpy as np

import anomapy.train.sssn as sssn
import pyworld.toolkit.tools.visutils.transform as T
import pyworld.toolkit.tools.visutils.jupyter as J
import pyworld.toolkit.tools.torchutils as tu

#load model
env = "Breakout"
run = "benedict-wilkins/anomapy/sssn-Breakout-20200209131829"
models, config = wbu.load(run)
config['state'] = dict(shape=config['state_shape'])
model = models['model.pt'].load(sssn.model(**config))
config = SimpleNamespace(**config)


#load anomaly data
import datasets
import numpy as np
dataset = datasets.dataset('aad.anomaly.{0}'.format(env))
dataset.state.transform.to_float().CHW().torch()
anoms = [(a,e[0]) for a,e in dataset.meta.anomaly.items()]
a_episodes = [d for d in dataset.state.load_files(*[e[1] for e in anoms])]
a_labels = [d for d in dataset.label.load_files(*[e[1] for e in anoms])]
a_tlabels = [np.logical_or(l[:-1], l[1:]).astype(np.uint8) for l in a_labels] #transition labels



USING DEVICE: cuda
 -- found local run at /home/ben/Documents/repos/anomaly-detection/wandb/benedict-wilkins/anomapy/sssn-Breakout-20200209131829
 -- found config file: 
 -- found 1 model(s): 
 ---- model.pt
loading dataset state...


IntProgress(value=0, max=7)

loading dataset label...


IntProgress(value=0, max=7)

# Traditional Visualisation

In [3]:
#visualise


a = 5
episode = a_episodes[a]
labels = a_labels[a].astype(np.uint8)

def plot_latent(model, episode):
    z = tu.to_numpy(sssn.encode(model, episode))
    images = T.HWC(tu.to_numpy(episode))
    return J.scatter_image(z[:,0], z[:,1], images, scatter_colour=np.array(['blue','red'])[labels], line_colour='#b9d1fa', scale=1.5)

from ipywidgets import Image, Layout, VBox, HBox, interact, IntSlider, IntProgress, HTML, Output
import ipywidgets as widgets

fig, image = plot_latent(model, episode)

fig.update_layout(width=600, height=500)
#box_layout = widgets.Layout(display='flex',flex_flow='row',align_items='center',width='100%',height='100%')
#display(HBox([fig], layout=box_layout))

#box_layout = widgets.Layout(display='flex',flex_flow='row',align_items='center',width='100%',height='100%')
#display(HBox([fig, image_widget], layout=box_layout)) #basically... this needs to be done in jupyter..?!]


FigureWidget({
    'data': [{'line': {'color': '#b9d1fa'},
              'marker': {'color': array(['blue', 'b…

# Distance Vector Visualisation

In [2]:
for a in range(len(anoms)-1):
    print(anoms[a][0])
    episode = a_episodes[a]
    labels = a_tlabels[a].astype(np.uint8)

    def plot_latent(model, episode):
        z = tu.to_numpy(sssn.encode(model, episode))
        images = T.HWC(tu.to_numpy(episode))
        d = z[:-1] - z[1:]

        print(d.shape, labels.shape)

        return J.scatter_image(d[:,0], d[:,1], images, mode=J.line_mode.marker, scatter_colour=np.array(['blue','red'])[labels], line_colour=None, scale=1.5)

    fig, image = plot_latent(model, episode)
    display(fig)
    path = "wandb/{0}/media/{1}-dvec-visualise.png".format(run,anoms[a][0])
    fig.write_image(path)


fill
(2513, 2) (2513,)


FigureWidget({
    'data': [{'marker': {'color': array(['blue', 'blue', 'blue', ..., 'blue', 'blue', 'blue'], …

block
(3009, 2) (3009,)


FigureWidget({
    'data': [{'marker': {'color': array(['blue', 'blue', 'blue', ..., 'blue', 'blue', 'blue'], …

freeze
(3570, 2) (3570,)


...

FigureWidget({
    'data': [{'marker': {'color': array(['blue', 'blue', 'blue', ..., 'blue', 'blue', 'blue'], …

split_horizontal
(3421, 2) (3421,)


FigureWidget({
    'data': [{'marker': {'color': array(['blue', 'blue', 'blue', ..., 'blue', 'blue', 'blue'], …

split_vertical
(1766, 2) (1766,)


FigureWidget({
    'data': [{'marker': {'color': array(['blue', 'blue', 'blue', ..., 'red', 'red', 'blue'], dt…